In [2]:
'''
hello im ab01fazl
ive written comprehensive comments for you
if you want to test the file just simply run every cell
and if you want to get the output uncomment the last line
happy hacking !!!
'''

import pandas as pd
from bs4 import BeautifulSoup
import requests

In [4]:
urls = ["https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Fellowship_of_the_Ring",
        'https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Two_Towers',
        'https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Return_of_the_King',
        'https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Rings_of_Power']


In [7]:
columns = ['title', 'actor', 'role']
df = pd.DataFrame(columns=columns)

# update df
def add_to_df(row):
        df.loc[len(df)] = row.values()


# i seperated this chunk of code into its own function (DRY rule)
def extract(cast):
        # loop the cast and get li`s and get the text before the ':'
        for i in cast:
                name = i.text.split(':')[0]
                splitted = name.split(' as ')

                # special cases that i couldnt handle with split
                if splitted[0] == 'Andy Serkis':
                        splitted[1] = 'Gollum'

                if splitted[0] == 'Joseph Mawle (season 1) and Sam Hazeldine (season 2)':
                        splitted[0] = 'Joseph Mawle'
                        df2 = {
                                'title': soup.find(id='firstHeading').text,
                                'actor': 'Sam Hazeldine',
                                'role': 'Adar'
                        }
                        add_to_df(df2)

                # and the standard cases
                if len(splitted) == 2:
                        df2 = {
                                'title': soup.find(id='firstHeading').text,
                                'actor': splitted[0],
                                'role': splitted[1]
                        }
                        add_to_df(df2)

for u in urls:
        page = requests.get(u)
        soup = BeautifulSoup(page.content, "html.parser")

        # get the cast element
        # in the rings of power the element id is different
        if u == urls[-1]:
                cast = soup.find(id='Cast_and_characters').parent.findNext('ul').findAll('li')
        else:
                cast = soup.find(id='Cast').parent.findNext('ul').findAll('li')

        extract(cast)

# the cast also includes in the fellowship of the ring
page = requests.get(urls[0])
soup = BeautifulSoup(page.content, "html.parser")
also_cast = soup.find(id='Cast').parent.findNext('ul').find_next('ul')
extract(also_cast)


# df
df.to_csv('lotr_casts.csv', index=False)